In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from owi_data_2_pandas.io import API
import pandas as pd
import os
import datetime

import dotenv
dotenv.load_dotenv()

True

In [10]:
# Ensure connection to the VPN
user = os.getenv('API_USERNAME') 
password = os.getenv('API_PASSWORD')
root = r"http://10.0.0.138/api/v1/"

In [26]:
location = 'nw2c02'
owi_api = API(api_root=root, username=user, password=password)
owi_api.groups

,project,locations,datagroups
id,,,
1,prj_smartbridge,"[mo01, opzuid]","[acceleration, fiber, mpe]"
2,parkwind_northwester2,"[nw2e01, nw2f01, nw2d01]",[scada]
3,parkwind,[bbc01],"[acceleration, mpe, scada]"
4,parkwind_northwester2,"[nw2e01, nw2f01, nw2d01, nw2a01, nw2a02, nw2a0...","[acceleration, mpe, scada]"
5,nobelwind,"[bbg10, bbk05, bbg08]","[FBG, acceleration, bolt, corrosion, fatigue, ..."


In [ ]:
metrics = owi_api.metrics(locations=location)
metrics

In [13]:
# Get small data set
start = datetime.datetime(2023, 6, 1, tzinfo=datetime.timezone.utc)
stop = datetime.datetime(2023, 6, 10, tzinfo=datetime.timezone.utc)
small_data = owi_api.query((start, stop), location=location, metrics = metrics.metric)
small_data

,location,mean_NW2_C02_rpm,mean_NW2_C02_yaw,std_NW2_C02_pitch,mean_NW2_C02_pitch,mean_NW2_C02_power,std_NW2_C02_windspeed,mean_NW2_C02_windspeed,mean_NW2_C02_winddirection,mpe_NW2_C02_FA,...,min_NW2_C02_NAC_ACC_SS,rms_NW2_C02_NAC_ACC_FA,rms_NW2_C02_NAC_ACC_SS,std_NW2_C02_NAC_ACC_FA,std_NW2_C02_NAC_ACC_SS,mean_NW2_C02_NAC_ACC_FA,mean_NW2_C02_NAC_ACC_SS,rms1p_NW2_C02_NAC_ACC_Z,rms1p_NW2_C02_NAC_ACC_FA,rms1p_NW2_C02_NAC_ACC_SS
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-06-01 00:00:00+00:00,NW2C02,10.445,16.338,0.831241,-3.525,8118.208,0.754,11.391,17.150,"[{'size': 32, 'algorithm': 'lscf', 'std_dampin...",...,-0.036635,0.010305,0.011513,0.010305,0.011513,0.0,0.0,0.000431,0.003753,0.004629
2023-06-01 00:10:00+00:00,NW2C02,10.445,20.535,1.015521,-2.854,6843.520,1.023,10.609,22.353,"[{'size': 16, 'algorithm': 'lscf', 'std_dampin...",...,-0.033817,0.009921,0.010211,0.009921,0.010211,0.0,0.0,0.000430,0.003918,0.003613
2023-06-01 00:20:00+00:00,NW2C02,10.445,24.843,0.709950,-2.070,5771.776,1.277,10.180,24.032,"[{'size': 28, 'algorithm': 'lscf', 'std_dampin...",...,-0.036501,0.010950,0.010467,0.010950,0.010467,0.0,0.0,0.000427,0.003839,0.004551
2023-06-01 00:30:00+00:00,NW2C02,10.445,25.179,0.956972,-2.826,6920.320,1.156,11.129,24.060,"[{'size': 33, 'algorithm': 'lscf', 'std_dampin...",...,-0.038860,0.011446,0.011084,0.011446,0.011084,0.0,0.0,0.000452,0.004023,0.004891
2023-06-01 00:40:00+00:00,NW2C02,10.445,24.955,0.644423,-2.042,5688.768,0.922,10.141,25.095,"[{'size': 41, 'algorithm': 'lscf', 'std_dampin...",...,-0.042127,0.011552,0.011688,0.011552,0.011688,0.0,0.0,0.000444,0.003902,0.004729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-09 23:20:00+00:00,NW2C02,9.251,76.711,0.049877,-1.063,2817.408,0.512,8.223,76.068,"[{'size': 34, 'algorithm': 'lscf', 'std_dampin...",...,-0.025329,0.004662,0.008753,0.004662,0.008753,0.0,0.0,0.000061,0.000605,0.001408
2023-06-09 23:30:00+00:00,NW2C02,9.251,76.739,0.050711,-1.035,2868.224,0.434,8.055,75.480,"[{'size': 26, 'algorithm': 'lscf', 'std_dampin...",...,-0.031668,0.005123,0.009830,0.005123,0.009830,0.0,0.0,0.000074,0.000669,0.001381
2023-06-09 23:40:00+00:00,NW2C02,9.848,76.739,0.051341,-1.063,3435.200,0.445,8.793,75.620,"[{'size': 41, 'algorithm': 'lscf', 'std_dampin...",...,-0.028145,0.004344,0.009848,0.004344,0.009848,0.0,0.0,0.000099,0.000984,0.002696


In [15]:
# Get a bigger dataset
# For this we will need a halper function as the API has a hard time collecting more than 15 days of data for all metrics

In [8]:
def get_big_data(start, end, owi_api, location, metrics, days=15):
    new_start = start
    all_data = pd.DataFrame()
    middle = start + datetime.timedelta(days=days)
    while end > middle:
        data_ = owi_api.query((new_start, middle), location=location, metrics = metrics)
        new_start = middle
        middle = new_start + datetime.timedelta(days=days)
        all_data = pd.concat([all_data, data_])
    if end < middle:
        data_ = owi_api.query((new_start, end), location=location, metrics = metrics)
        all_data = pd.concat([all_data, data_])
    duplicated_indices = all_data.index.duplicated(keep='first')
    all_data_unique = all_data[~duplicated_indices]
    return all_data_unique

In [16]:
start = datetime.datetime(2023, 10, 1, tzinfo=datetime.timezone.utc)
stop = datetime.datetime(2024, 1, 1, tzinfo=datetime.timezone.utc)
big_data = get_big_data(start, stop, owi_api, location, metrics.metric, days=10)
big_data

,location,mean_NW2_C02_rpm,mean_NW2_C02_yaw,std_NW2_C02_pitch,mean_NW2_C02_pitch,mean_NW2_C02_power,std_NW2_C02_windspeed,mean_NW2_C02_windspeed,mean_NW2_C02_winddirection,mpe_NW2_C02_FA,...,min_NW2_C02_NAC_ACC_SS,rms_NW2_C02_NAC_ACC_FA,rms_NW2_C02_NAC_ACC_SS,std_NW2_C02_NAC_ACC_FA,std_NW2_C02_NAC_ACC_SS,mean_NW2_C02_NAC_ACC_FA,mean_NW2_C02_NAC_ACC_SS,rms1p_NW2_C02_NAC_ACC_Z,rms1p_NW2_C02_NAC_ACC_FA,rms1p_NW2_C02_NAC_ACC_SS
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-01 00:00:00+00:00,NW2C02,7.759,196.898,0.113602,-1.063,1779.392,0.598,6.852,196.311,"[{'size': 32, 'algorithm': 'lscf', 'std_dampin...",...,-0.024618,0.007709,0.008589,0.007709,0.008589,0.0,0.0,0.000029,0.000268,0.000144
2023-10-01 00:10:00+00:00,NW2C02,7.759,196.870,0.132962,-1.035,1697.024,0.734,6.824,197.766,"[{'size': 33, 'algorithm': 'lscf', 'std_dampin...",...,-0.028772,0.007504,0.008445,0.007504,0.008445,0.0,0.0,0.000027,0.000260,0.000161
2023-10-01 00:20:00+00:00,NW2C02,7.460,196.842,0.117619,-1.063,1548.096,0.707,6.555,198.073,"[{'size': 30, 'algorithm': 'lscf', 'std_dampin...",...,-0.028088,0.007223,0.008300,0.007223,0.008300,0.0,0.0,0.000022,0.000205,0.000143
2023-10-01 00:30:00+00:00,NW2C02,6.864,202.634,0.320148,-0.951,1082.432,0.668,5.848,206.103,"[{'size': 37, 'algorithm': 'lscf', 'std_dampin...",...,-0.028432,0.008611,0.008196,0.008611,0.008196,0.0,0.0,0.000018,0.000159,0.000091
2023-10-01 00:40:00+00:00,NW2C02,6.864,210.271,0.340952,-0.923,1090.880,0.762,5.992,212.090,"[{'size': 16, 'algorithm': 'lscf', 'std_dampin...",...,-0.028819,0.008911,0.007697,0.008911,0.007697,0.0,0.0,0.000015,0.000137,0.000086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 23:20:00+00:00,NW2C02,10.445,219.307,2.542477,11.219,9524.800,2.199,17.965,219.503,"[{'size': 23, 'algorithm': 'lscf', 'std_dampin...",...,-0.040263,0.010900,0.010199,0.010900,0.010199,0.0,0.0,0.000444,0.004147,0.004286
2023-12-31 23:30:00+00:00,NW2C02,10.445,224.399,1.405813,12.505,9524.928,1.527,18.660,224.119,"[{'size': 32, 'algorithm': 'lscf', 'std_dampin...",...,-0.028548,0.011222,0.008155,0.011222,0.008155,0.0,0.0,0.000477,0.004368,0.003383
2023-12-31 23:40:00+00:00,NW2C02,10.445,226.553,1.605133,11.414,9525.056,1.598,17.832,225.798,"[{'size': 36, 'algorithm': 'lscf', 'std_dampin...",...,-0.031743,0.011545,0.009464,0.011545,0.009464,0.0,0.0,0.000465,0.004361,0.003832


In [17]:
# Save the data locally
# big_data_path = ...
# big_data.to_parquet(big_data_path)